In [6]:
import pandas as pd
import numpy as np

In [ ]:
import pickle
with open("df.pkl", "rb") as f:
    df = pickle.load(f)

df.head()           

,Company,TypeName,Ram,Weight,Price,Touchscreen,IPS,ppi,CPU_brand,Gpu_brand,os,HDD,SDD
0,Apple,Ultrabook,8,1.37,71378.6832,0,1,226.983005,Intel Core i5,Intel,MAc,0,128
1,Apple,Ultrabook,8,1.34,47895.5232,0,0,127.677940,Intel Core i5,Intel,MAc,0,0
2,HP,Notebook,8,1.86,30636.0000,0,0,141.211998,Intel Core i5,Intel,Other/No OS/Linux,0,256
3,Apple,Ultrabook,16,1.83,135195.3360,0,1,220.534624,Intel Core i7,AMD,MAc,0,512
4,Apple,Ultrabook,8,1.37,96095.8080,0,1,226.983005,Intel Core i5,Intel,MAc,0,256


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [11]:
x=df.drop(columns=["Price"])
y=np.log(df["Price"])

In [12]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15,random_state=2)

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [14]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
# from xgboost import XGBRFRegressor

In [20]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [21]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(alpha=0.001),
    "Ridge": Ridge(alpha=10),
    "K-Neighbors Regressor": KNeighborsRegressor(n_neighbors=4),
    "Decision Tree": DecisionTreeRegressor(max_depth=8),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=200,max_features="sqrt",max_depth=20,min_samples_leaf=1,min_samples_split=2),
    "ExtraTreesRegressor": ExtraTreesRegressor(n_estimators=100,max_features="sqrt",max_depth=20,min_samples_leaf=1,min_samples_split=2),
    "AdaBoost Regressor": AdaBoostRegressor(n_estimators=15,learning_rate=1.0)
}

In [ ]:
model_list = []
r2_list =[]
step1=ColumnTransformer(transformers=[("col_tnf",OneHotEncoder(sparse_output=False,drop="first"),[0,1,7,8,9])],remainder="passthrough")

for i in range(len(list(models))):
    model = list(models.values())[i]
    pipe=Pipeline([("step1",step1),("step2",model)])
    pipe.fit(x_train,y_train)

    # Make predictions
    y_train_pred = pipe.predict(x_train)
    y_test_pred = pipe.predict(x_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

ValueError: could not convert string to float: 'Toshiba'

In [ ]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
